# Extracting sign distributions

This notbook contains code for:
- importing necessary libraries
- loading stored WTs
- creating random sparse networks
- Extracting sign distributions from WTs
- Extracting sign distributions from non-WTs
- saving extracted distributions in files

### Requirements:

In [1]:
# importing necessary libraries and the cnn architecture I defined

from cnn_architecture import CNN2Model
from utils import *

import tensorflow_datasets as tfds
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [2]:
def load_and_prep_cifar(batch_size, shuffle_size):
    # load data set
    (train_ds, test_ds), ds_info = tfds.load(name="cifar10", split=["train","test"], as_supervised=True, with_info=True)
    # tfds.show_examples(train_ds, ds_info)
    
    def prepare_cifar10_data(ds):
        #convert data from uint8 to float32
        ds = ds.map(lambda img, target: (tf.cast(img, tf.float32), target))
        #sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
        ds = ds.map(lambda img, target: ((img/128.)-1., target))
        #create one-hot targets
        ds = ds.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
        #cache this progress in memory, as there is no need to redo it; it is deterministic after all
        ds = ds.cache()
        #shuffle, batch, prefetch
        ds = ds.shuffle(shuffle_size).batch(batch_size).prefetch(2)
        #return preprocessed dataset
        return ds
    
    # prepare data
    train_dataset = train_ds.apply(prepare_cifar10_data)
    test_dataset = test_ds.apply(prepare_cifar10_data)
    
    return train_dataset, test_dataset

In [3]:
# extract sign distribution from one layer

def sign_distribution_layer(this_layer, next_layer):
    '''
    columns: 
    ["prune_rate_in", "prune_rate_out", "pos_in", "pos_out", "neg_in", "neg_out", "sign_rate_in", "sign_rate_out"]
    '''
    sign_distr = pd.DataFrame()
    for this_neuron, next_neurons, i in zip(this_layer.T, next_layer, range(np.shape(this_layer)[0])):
        
        this_unpruned = this_neuron[this_neuron != 0.0]
        next_unpruned = next_neurons[next_neurons != 0.0]
        dic = {} 
        dic["pos_in"] = len(this_unpruned[this_unpruned>0])
        dic["pos_out"] = len(next_unpruned[next_unpruned>0])
        dic["neg_in"] = len(this_unpruned[this_unpruned<0])
        dic["neg_out"] = len(next_unpruned[next_unpruned<0])
        if len(this_neuron) > 0:
            dic["prune_rate_in"] = 1 - (len(this_unpruned)/len(this_neuron))
        else:
            dic["prune_rate_in"] = 1
        if len(next_unpruned) > 0:
            dic["prune_rate_out"] = 1 - (len(next_unpruned)/len(next_neurons))
        else:    
            dic["prune_rate_out"] = 1
        if len(this_unpruned) > 0:
            dic["sign_rate_in"] = dic["pos_in"]/len(this_unpruned)
        else:
            dic["sign_rate_in"] = 0
        if len(next_unpruned) > 0:    
            dic["sign_rate_out"] = dic["pos_out"]/len(next_unpruned)
        else:
            dic["sign_rate_out"] = 0                                
        df = pd.DataFrame(data = dic, index = [i])
        sign_distr = pd.concat([sign_distr, df], axis=0)
        
    return sign_distr

In [4]:
# extract sign distribution form two layers

def sign_distribution_layers(layer1, layer2, layer3):
    sign_distr1 = sign_distribution_layer(layer1, layer2)
    sign_distr2 = sign_distribution_layer(layer2, layer3)
    sign_distr1["layer"] = "dense1"
    sign_distr2["layer"] = "dense2"
    return pd.concat([sign_distr1, sign_distr2], axis=0)

## Extracting sign distributions and storing them as files:

In [5]:
# extract sign distributions

def get_sign_distr(dataset, pruning_name, n):

    # make a model to load the weights into
    train_dataset, test_dataset = load_and_prep_cifar(batch_size=60, shuffle_size=512)
    model = CNN2Model()
    model(list(train_dataset)[0][0])
    for i in range(n):
        
        # get WT weights
        model.load_weights(f"1b WTs/WT_{dataset}_{pruning_name}_{i}")    
        weights_wt = model.get_weights()
        # get sign distribution
        sign_distr_wt = sign_distribution_layers(weights_wt[4], weights_wt[6], weights_wt[8])
        # store sign distribution
        sign_distr_wt.to_csv(f'2b Sign distributions/{dataset}_{pruning_name}_{i}_sign_distr.csv', index=False)

In [6]:
get_sign_distr("SVHN","IMP",10)
get_sign_distr("CINIC","IMP",10)
get_sign_distr("CIFAR","IMP",10)

### Generating randomly pruned sign distributions

In [7]:
# implement random pruning of a weight matrix according to a pruning rate

def random_pruning(model,pruning_rates):
    
    weights = model.get_weights()
    new_weights = []
    
    for layer, p_rate in zip(weights[::2], pruning_rates):
        shape = np.shape(layer)
        layer = layer.flatten()
        number_to_prune = int(len(layer) * p_rate)
        pruning_indexi = random.sample(range(len(layer)),number_to_prune)
        layer[pruning_indexi] = 0.0
        layer = np.reshape(layer,shape)
        new_weights.append(layer)
        
    weights[::2] = new_weights
    model.set_weights(weights)
    weights = model.get_weights()
    
    return model

In [8]:
# get pruning rates from imp model
train_dataset, test_dataset = load_and_prep_cifar(batch_size=60, shuffle_size=512)
model_cifar_imp = CNN2Model()
model_cifar_imp(list(train_dataset)[0][0])
model_cifar_imp.load_weights("1b WTs/WT_CIFAR_IMP_0")
pruning_rates = get_pruning_rates(model_cifar_imp.get_weights()[::2])

# generate random sparse networks
for i in range(10):
    random_model = CNN2Model()
    random_model(list(train_dataset)[0][0])
    random_model = random_pruning(random_model,pruning_rates)
    weights_rsn = random_model.get_weights()
    sign_distr_random = sign_distribution_layers(weights_rsn[4], weights_rsn[6], weights_rsn[8])
    sign_distr_random.to_csv(f"2b Sign distributions/RSN_{i}_sign_distr.csv", index=False)

### Inspecting the data

In [9]:
# return the pruning rates of the conv and dense layers
print("conv rate: ", get_pruning_rate(model_cifar_imp.get_weights()[0:4:2]))
print("dense rate: ", get_pruning_rate(model_cifar_imp.get_weights()[4::2]))

conv rate:  0.6794931592039801
dense rate:  0.9176456925675676


In [10]:
sign_distr_random.head() 

,pos_in,pos_out,neg_in,neg_out,prune_rate_in,prune_rate_out,sign_rate_in,sign_rate_out,layer
0,564,97,567,89,0.930969,0.273438,0.498674,0.521505,dense1
1,588,95,576,97,0.928955,0.250000,0.505155,0.494792,dense1
2,574,98,592,97,0.928833,0.238281,0.492281,0.502564,dense1
3,589,85,579,100,0.928711,0.277344,0.504281,0.459459,dense1
4,592,112,580,90,0.928467,0.210938,0.505119,0.554455,dense1


In [11]:
sign_distr_random.tail()  

,pos_in,pos_out,neg_in,neg_out,prune_rate_in,prune_rate_out,sign_rate_in,sign_rate_out,layer
251,97,3,98,4,0.238281,0.3,0.497436,0.428571,dense2
252,99,5,90,3,0.261719,0.2,0.523810,0.625000,dense2
253,89,4,97,5,0.273438,0.1,0.478495,0.444444,dense2
254,94,5,95,4,0.261719,0.1,0.497354,0.555556,dense2
255,92,6,94,4,0.273438,0.0,0.494624,0.600000,dense2
